In [9]:
import pandas as pd
import pymssql
from sqlalchemy import create_engine

from sklearn.preprocessing import QuantileTransformer
from joblib import load

qt = QuantileTransformer(output_distribution="normal")
pipe = load("files/linear_model.joblib")


In [10]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

In [11]:
df = pd.read_sql(
    """
    select ticker, roeq, bm, mom12m, mve, famaindustry
    from today
    """, 
    conn
)
df = df.dropna()
df = df.set_index("ticker")

In [ ]:
df = df.sort_values(by="mve")
df = df.iloc[:-500]

In [ ]:
features = ["roeq", "bm", "mom12m"]
df[features] = qt.fit_transform (df[features])
df["predict"] = pipe.predict(df[features])
df.to_csv("files/predictions_2023-01-20.csv")